In [3]:
import open3d as o3d
import numpy as np


pcd = o3d.io.read_point_cloud("../merged_aligned1.ply")

In [12]:
# Step 1: Statistical outlier removal
pcd_clean, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.05)

# Step 2: Downsample
pcd_down = pcd_clean.voxel_down_sample(voxel_size=0.02)

# Step 3: Estimate normals (required for some clustering tools)
pcd_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

# Step 4: Cluster based on density (DBSCAN)
labels = np.array(pcd_down.cluster_dbscan(eps=0.03, min_points=20))
max_label = labels.max()
print(f"Found {max_label + 1} clusters.")

# Step 5: Get the biggest cluster (assumed to be the detail)
counts = [(i, np.sum(labels == i)) for i in range(max_label + 1)]
best_label = max(counts, key=lambda x: x[1])[0]
detail = pcd_down.select_by_index(np.where(labels == best_label)[0])

# Step 6: Bounding box around it
aabb = detail.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)

# Visualize
o3d.visualization.draw_geometries([pcd_down, aabb], window_name="Auto-segmented object")

Found 7 clusters.


In [15]:
import time
# Step 1: Remove outliers
pcd_clean, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.005)

# Step 2: Downsample
pcd_down = pcd_clean.voxel_down_sample(voxel_size=0.02)

# Step 3: Estimate normals
pcd_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

# Step 4: DBSCAN clustering
labels = np.array(pcd_down.cluster_dbscan(eps=0.03, min_points=20, print_progress=True))
n_clusters = labels.max() + 1
print(f"Found {n_clusters} clusters")

# Step 5: Visualize clusters one by one
for i in range(n_clusters):
    cluster = pcd_down.select_by_index(np.where(labels == i)[0])
    cluster.paint_uniform_color(np.random.rand(3))
    print(f"Showing cluster {i}, points: {len(cluster.points)}")
    o3d.visualization.draw_geometries([cluster], window_name=f"Cluster {i}")
    time.sleep(0.5)  # pause if running in loop

Found 8 clustersbors.[=>                                      ] 2%
Showing cluster 0, points: 20
Precompute neighbors.[========================================] 100%
Showing cluster 1, points: 26                      ] 2%
Showing cluster 2, points: 20
Showing cluster 3, points: 29
Showing cluster 4, points: 21
Showing cluster 5, points: 21
Showing cluster 6, points: 22
Showing cluster 7, points: 17
